Here I take either the sarc_tweets_2_4 dataset or the not_sarc_with_emojis dataset, pharse and clean them, and save as sarc_emojis_1000 or not_sarc_emojis_1000 datasets 

In [ ]:
! gdown 1GHD3RuN6NaSYjQWw16UAvrNhl9fBOkpG
! gdown 1yoB2WZwW3YIefSqaS-ThsBuXN-unH9_S
! pip install emoji

Downloading...
From: https://drive.google.com/uc?id=1GHD3RuN6NaSYjQWw16UAvrNhl9fBOkpG
To: /content/sarc_tweets_2_4.csv
100% 291M/291M [00:06<00:00, 46.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yoB2WZwW3YIefSqaS-ThsBuXN-unH9_S
To: /content/not_sarc_with_emojis2.csv
100% 187M/187M [00:03<00:00, 51.9MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 6.7 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=133db8dd135ff40cc5feced49e252f463964907616c235ef6eae61fd29cbab23
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
import re
import emoji

def extract_emojis(s):
  return [d['emoji'] for d in emoji.emoji_list(s)]


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def clean_com(com):
  com = com.lower()
  com = re.sub(r'http\S+', '', com)
  com = re.sub("@[A-Za-z0-9_]+","", com)
  com = re.sub("#[A-Za-z0-9_]+","", com)
  com = re.sub("rt :","", com)
  com = re.sub("^ +","", com)
  com = re.sub(" +$","", com)
  com = re.sub("\n","", com)
  com = re.sub(" +"," ", com)
  return com

In [ ]:
emojies = ['😒','😂','👍','👌','🙄','😑','😊','😉','😏','😁','😔','😡','😐','😳','😜','😄','👏','☺','🤔','😃']

In [ ]:
import pandas as pd
data = pd.read_csv('not_sarc_with_emojis2.csv') #sarc_tweets_2_4.csv

tweets = data['text']
print("At first:", len(tweets))

tweets = list(dict.fromkeys(tweets))
print("After removing duplicates:",len(tweets))

tweets = [x for x in tweets if ('#sarcasm' not in x) or ('#sarcastic' not in x)]
print("with #sarcasm and #sarcastic:", len(tweets))

tweets = [clean_com(x) for x in tweets]

tweets = [x for x in tweets if ('sarcasm' not in x) and ('sarcastic' not in x)]
print("Tweets without sarcasm or sarcastic:", len(tweets))

tweets = [x for x in tweets if len(extract_emojis(x)) == 1]
print("Tweets with 1 emoji:", len(tweets))

tweets = [x for x in tweets if any(e in x for e in emojies)]
print("tweets with one of 20 emojies", len(tweets))

At first: 1480211
After removing duplicates: 1474545
with #sarcasm and #sarcastic: 1474545
Tweets without sarcasm or sarcastic: 1473855
Tweets with 1 emoji: 767428
tweets with one of 20 emojies 767428


In [ ]:
tweets1 = [x for x in tweets if 20 <= len(x) <= 512]
print("tweets in size", len(tweets1))

tweets in size 607099


In [ ]:
my_dict = {}
for t in tweets1:
  e = extract_emojis(t)[0]
  if e not in my_dict:
    my_dict[e] = [remove_emoji(t)]
  elif len(my_dict[e]) < 1000:
    my_dict[e].append(remove_emoji(t))

In [ ]:
for e in emojies:
  print(e,len(my_dict[e]))

😒 1000
😂 1000
👍 1000
👌 1000
🙄 1000
😑 1000
😊 1000
😉 1000
😏 1000
😁 1000
😔 1000
😡 1000
😐 1000
😳 1000
😜 1000
😄 1000
👏 1000
☺ 1000
🤔 1000
😃 1000


In [ ]:
import csv
from google.colab import drive

drive.mount('/content/gdrive')
with open('/content/gdrive/My Drive/not_sarc_emojis_1000.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(["label", "text"])
  for e in emojies:
    for t in my_dict[e]:
      writer.writerow([e,t]) 


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
